In [3]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
import json
from pprint import pprint


In [5]:
#Extract CSV into DataFrame
prime_file = "../Resources/amazon_prime_movies.csv"
prime_df = pd.read_csv(prime_file)
prime_df.head()

,movie_name,language,imdb_rating,running_time,year_of_release,maturity_rating,plot
0,John Rambo,Tamil,7,1 h 26 min,2008,18+,"In Thailand, John Rambo joins a group of merce..."
1,American Pie,English,7,1 h 35 min,1999,18+,You'll never look at warm apple pie the same w...
2,Bombshell,English,6.8,1 h 48 min,2019,18+,"The provocative real story of three smart, amb..."
3,Love Birds,Tamil,5.1,2 h 40 min,1996,All,"After accidentally meeting, Mridula (Nagma) an..."
4,Hippi,Telugu,5,2 h 20 min,2019,18+,Hippi is about the live-in relationship betwee...


In [64]:
#Creating new Dataset with selected coloumns and clean it
new_prime_df = prime_df[['movie_name', 'language', 'imdb_rating','maturity_rating']].copy()
new_prime_df.head()

new_prime_df = prime_df[prime_df['imdb_rating'].str.strip() != 'None']
new_prime_df['imdb_rating']

0         7
1         7
2       6.8
3       5.1
4         5
       ... 
8121    6.7
8122    5.2
8123    6.5
8124    7.1
8126      4
Name: imdb_rating, Length: 6119, dtype: object

In [65]:
#Transform Prime DataFrame
#Create a filtered dataframe from specific columns
prime_cols = ["movie_name", "language", "imdb_rating", "maturity_rating"]
prime_transformed = new_prime_df[prime_cols].copy()

# Rename the column headers
prime_transformed = prime_transformed.rename(columns={"movie_name": "movie_name",
                                                          "language": "language",
                                                          "imdb_rating": "imdb_rating",
                                                        "maturity_rating": "maturity_rating"})

# Clean the data by dropping duplicates and setting the index
prime_transformed.drop_duplicates("movie_name", inplace=True)
prime_transformed.set_index("imdb_rating", inplace=True)

prime_transformed.head()


,movie_name,language,maturity_rating
imdb_rating,,,
7,John Rambo,Tamil,18+
7,American Pie,English,18+
6.8,Bombshell,English,18+
5.1,Love Birds,Tamil,All
5,Hippi,Telugu,18+


In [66]:
#Create Database connection
connection_string = "postgres:postgres@localhost:5432/prime1_db"
engine = create_engine(f'postgresql://{connection_string}')

In [67]:
# Confirm tables
engine.table_names()

/var/folders/s3/_tx_vh_n3jxcv5xmm_5v0lpr0000gn/T/ipykernel_11639/3015872754.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['prime', 'prime_table']

In [68]:
#Use pandas to Load  csv converted DataFrames into database
prime_transformed.to_sql(name='prime_table', con=engine, if_exists='append', index=True)

In [69]:
#confirm data has been added by quering the table
pd.read_sql_query('select * from prime_table', con=engine).head()

,movie_name,language,imdb_rating,running_time,year_of_release,plot,maturity_rating
0,John Rambo,Tamil,7.0,None,None,None,18+
1,American Pie,English,7.0,None,None,None,18+
2,Bombshell,English,6.8,None,None,None,18+
3,Love Birds,Tamil,5.1,None,None,None,All
4,Hippi,Telugu,5.0,None,None,None,18+
